# Project

# Imports

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt  # Corrected spelling
import seaborn as sns  # For enhanced data visualization
from scipy import stats  # For statistical tests
import warnings  # To suppress warnings during EDA
import re
warnings.filterwarnings("ignore")  # Optional, to make output cleaner

## Load Data

In [ ]:
df_raw = pd.read_excel("data_for_analysis.xlsx")
df_raw.head(3)


In [19]:
df = pd.DataFrame({})

## Convert numeric columns

In [26]:
df['numeric_key'] = df_raw['key'].str.split('-').str[1].astype(int)
df['fields.Analysis'] = df_raw['fields.Analysis'].astype(str)


In [23]:
# # Function to extract numeric part from outwardIssue or inwardIssue
# def extract_nccfm_key(issue_links):
#     if issue_links and isinstance(issue_links, list):  # Check if not None and is a list
#         for link in issue_links:
#             # Check outwardIssue
#             if 'outwardIssue' in link and 'key' in link['outwardIssue']:
#                 match = re.search(r'NCCFM-(\d+)', link['outwardIssue']['key'])
#                 if match:
#                     return int(match.group(1))  # Extract numeric part
            
#             # Check inwardIssue
#             if 'inwardIssue' in link and 'key' in link['inwardIssue']:
#                 match = re.search(r'NCCFM-(\d+)', link['inwardIssue']['key'])
#                 if match:
#                     return int(match.group(1))  # Extract numeric part
#     return None  # Return None if no match

# # Load the data
# df_raw = pd.read_excel("data_for_analysis.xlsx")

# # Apply the function to extract the numeric keys
# df_raw['extracted_key'] = df_raw['fields.issuelinks'].apply(extract_nccfm_key)

In [27]:
df.head(10)

,numeric_key,extracted_key,fields.issuelinks
0,10561,None,[]
1,10560,None,[]
2,10559,None,[]
3,10558,None,[]
4,10557,None,[]
5,10556,None,[]
6,10555,None,"[{'id': '14232186', 'self': 'https://jiradc2.e..."
7,10554,None,[]
8,10553,None,[]
9,10552,None,[]


## Convert Categorical columns

## Convert Oridinal columns
